In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging
import shutil
from omegaconf import OmegaConf
from hydra import initialize, compose
from hydra.core.config_store import ConfigStore
import mlflow
from src.experiments.sner import sner
from pathlib import Path

import os

logging.basicConfig(
    format="%(asctime)s %(levelname)s:%(message)s",
    level=logging.INFO,
    datefmt="%I:%M:%S",
)
logger = logging.getLogger("training")

In [3]:
sentence_length = 110

run_experiments = os.getenv("UVL_BERT_RUN_EXPERIMENTS","True") == "FALSE"
pin_commits = os.getenv("UVL_BERT_PIN_COMMITS","True") == "FALSE"

print(f"{run_experiments=}")
print(f"{pin_commits=}")

run_experiments=True
pin_commits=True


In [4]:
from tooling.config import Experiment, Transformation

base_experiment_config = Experiment(
    name="Base Config", iterations=1, force=False
)

levels_transformation_config = Transformation(
    description="Levels",
    type="Reduced",
    task="Domain_Level",
    domain_data="Domain_Level",
    activity="Domain_Level",
    stakeholder="Domain_Level",
    system_function="Interaction_Level",
    interaction="Interaction_Level",
    interaction_data="Interaction_Level",
    workspace="Interaction_Level",
    software="System_Level",
    internal_action="System_Level",
    internal_data="System_Level",
)

label_transformation_config = Transformation(
    description="None",
    type="Full",
    task="Task",
    domain_data="Domain_Data",
    activity="Activity",
    stakeholder="Stakeholder",
    system_function="System_Function",
    interaction="Interaction",
    interaction_data="Interaction_Data",
    workspace="Workspace",
    software="System_Level",
    internal_action="System_Level",
    internal_data="System_Level",
)

In [5]:
from tooling.transformation import get_hint_transformation
import pickle

hint_transformation = get_hint_transformation(
    transformation_cfg=OmegaConf.structured(levels_transformation_config)
)
hint_label2id = hint_transformation["label2id"]
pickle.dump(
    hint_label2id, open("./src/service/models/hint_label2id.pickle", "wb")
)
hint_id2label = {y: x for x, y in hint_label2id.items()}
pickle.dump(
    hint_id2label, open("./src/service/models/hint_id2label.pickle", "wb")
)

transformation = get_hint_transformation(
    transformation_cfg=OmegaConf.structured(label_transformation_config)
)
label2id = transformation["label2id"]
pickle.dump(label2id, open("./src/service/models/label2id.pickle", "wb"))
id2label = {y: x for x, y in label2id.items()}
pickle.dump(id2label, open("./src/service/models/id2label.pickle", "wb"))

03:01:57 INFO:Hint Label2Id: hint_label2id={'0': 0, 'Domain_Level': 1, 'Interaction_Level': 2, 'System_Level': 3}
03:01:57 INFO:Hint Label2Id: hint_label2id={'0': 0, 'Task': 1, 'Domain_Data': 2, 'Activity': 3, 'Stakeholder': 4, 'System_Function': 5, 'Interaction': 6, 'Interaction_Data': 7, 'Workspace': 8, 'System_Level': 9}


## Train BiLSTM First Stage Model

In [6]:
from tooling.observability import get_run_id
from tooling.config import BiLSTMConfig, BiLSTM

from copy import deepcopy

bilstm_experiment_config = deepcopy(base_experiment_config)
bilstm_experiment_config.name = "Production: BiLSTM Train for Staged Application"

bilstm_config = BiLSTM(sentence_length=sentence_length)

bilstm_cfg: BiLSTMConfig = OmegaConf.structured(
    BiLSTMConfig(
        bilstm=bilstm_config,
        experiment=bilstm_experiment_config,
        transformation=levels_transformation_config,
    )
)

#bilstm_cfg.experiment.force = True

if run_experiments:
    from experiments.bilstm import bilstm
    bilstm(bilstm_cfg)


bilstm_run_id = get_run_id(bilstm_cfg, pin_commit = pin_commits)

print(bilstm_run_id)

print(mlflow.get_artifact_uri())

bilstm_run = mlflow.get_run(bilstm_run_id)
mlflow.artifacts.download_artifacts(
    f"{bilstm_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bilstm"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename("./src/service/models/bilstm")

03:01:59 INFO:
bilstm:
  type: BiLSTM
  sentence_length: 110
  batch_size: 32
  number_epochs: 4
  verbose: 1
  weighted_classes: false
  learning_rate: 0.0001
experiment:
  name: 'Production: BiLSTM Train for Staged Application'
  description: ''
  random_state: 125
  folds: 5
  iterations: 1
  average: macro
  dataset: prolific
  lower_case: false
  force: false
transformation:
  description: Levels
  type: Reduced
  task: Domain_Level
  goals: null
  domain_data: Domain_Level
  activity: Domain_Level
  stakeholder: Domain_Level
  system_function: Interaction_Level
  interaction: Interaction_Level
  interaction_data: Interaction_Level
  workspace: Interaction_Level
  software: System_Level
  internal_action: System_Level
  internal_data: System_Level

03:02:00 INFO:New experiment. Running
03:02:00 INFO:Entering mlflow context
03:02:01 INFO:Created a temporary directory at /var/folders/82/x_tg3xgx1px781gb8v4bny1r0000gn/T/tmpm5oqqqf7
03:02:01 INFO:Writing /var/folders/82/x_tg3xgx1px781

Epoch 1/4
34/34 [==============================] - 7s 179ms/step - loss: 1.2253 - precision: 0.3540 - recall: 0.3506 - val_loss: 1.0727 - val_precision: 0.3172 - val_recall: 0.3333
Epoch 2/4
34/34 [==============================] - 6s 170ms/step - loss: 0.7066 - precision: 0.3172 - recall: 0.3333 - val_loss: 0.2058 - val_precision: 0.3172 - val_recall: 0.3333
Epoch 3/4
34/34 [==============================] - 6s 173ms/step - loss: 0.1820 - precision: 0.3172 - recall: 0.3333 - val_loss: 0.1667 - val_precision: 0.3172 - val_recall: 0.3333
Epoch 4/4
34/34 [==============================] - 6s 177ms/step - loss: 0.1599 - precision: 0.3172 - recall: 0.3333 - val_loss: 0.1521 - val_precision: 0.3172 - val_recall: 0.3333


03:03:13 INFO:Assets written to: /Users/bockstaller/code/uvl-tore-classifier-bert/src/data/temp/bilstm/likeable-calf-64/0_model/assets
2023-08-15 15:03:22.876339: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-08-15 15:03:22.991861: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-08-15 15:03:23.004884: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-08-15 15:03:23.055586: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-08-15 15:03:23.067091: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - 1s 33ms/step


03:03:23 INFO:Logged iteration result res.precision=0.18282600526118 res.recall=0.25
03:03:23 INFO:Finished iteration=0
03:03:23 INFO:Breaking early after iteration=0 of 5 folds
03:03:24 INFO:Logged experiment result res.mean_precision=0.18282600526118 res.mean_recall=0.25
03:03:24 INFO:Left mlflow context


941f08130ff84d11bf2c80fc13790a81
mlflow-artifacts:/32/727a82c476c8414f84827ef1b521d28f/artifacts


PosixPath('src/service/models/bilstm')

## Train SNER First Stage Model

In [7]:
from tooling.observability import get_run_id
from tooling.config import SNERConfig, SNER

from copy import deepcopy

sner_experiment_config = deepcopy(base_experiment_config)
sner_experiment_config.name = "Production: SNER Train for Staged Application"

sner_config = SNER()

sner_cfg = OmegaConf.structured(
    SNERConfig(
        sner=sner_config,
        experiment=sner_experiment_config,
        transformation=levels_transformation_config,
    )
)

if run_experiments:
    from experiments.sner import sner
    sner(OmegaConf.create(sner_cfg))

sner_run_id = get_run_id(sner_cfg, pin_commit = pin_commits)
print(mlflow.get_artifact_uri())
print(sner_run_id)

sner_run = mlflow.get_run(sner_run_id)
mlflow.artifacts.download_artifacts(
    f"{sner_run.info.artifact_uri}/0_model.ser.gz",
    dst_path=Path("./src/service/models/"),
)
try:
    Path("./src/service/models/sner.ser.gz").unlink()
except FileNotFoundError:
    pass
Path("./src/service/models/0_model.ser.gz").rename(
    "./src/service/models/sner.ser.gz"
)

03:03:26 INFO:
sner:
  type: SNER
experiment:
  name: 'Production: SNER Train for Staged Application'
  description: ''
  random_state: 125
  folds: 5
  iterations: 1
  average: macro
  dataset: prolific
  lower_case: false
  force: false
transformation:
  description: Levels
  type: Reduced
  task: Domain_Level
  goals: null
  domain_data: Domain_Level
  activity: Domain_Level
  stakeholder: Domain_Level
  system_function: Interaction_Level
  interaction: Interaction_Level
  interaction_data: Interaction_Level
  workspace: Interaction_Level
  software: System_Level
  internal_action: System_Level
  internal_data: System_Level

03:03:27 INFO:New experiment. Running
03:03:27 INFO:Entering mlflow context
03:03:27 INFO:Importing dataset: prolific from /Users/bockstaller/code/uvl-tore-classifier-bert/src/data/datasets/prolific/TORE_Coded_Answers_1_33.json
03:03:27 INFO:Importing dataset: prolific from /Users/bockstaller/code/uvl-tore-classifier-bert/src/data/datasets/prolific/TORE_Coded_An

mlflow-artifacts:/32/727a82c476c8414f84827ef1b521d28f/artifacts
5ffff0522cb140cb85d6c0a402b2bd43


PosixPath('src/service/models/sner.ser.gz')

## Train BERT First Stage Model

In [8]:
from tooling.observability import get_run_id
from tooling.config import BERTConfig, BERT

from copy import deepcopy

bert_1_experiment_config = deepcopy(base_experiment_config)
bert_1_experiment_config.name = "Production: BERT_1 Train for Staged Application"

bert_1_config = BERT(max_len=sentence_length)

bert_1_cfg = OmegaConf.structured(
    BERTConfig(
        bert=bert_1_config,
        experiment=bert_1_experiment_config,
        transformation=levels_transformation_config,
    )
)

if run_experiments:
    from experiments.bert import bert
    bert(OmegaConf.create(bert_1_cfg))

bert_1_run_id = get_run_id(bert_1_cfg, pin_commit = pin_commits)

print(bert_1_run_id)

bert_1_run = mlflow.get_run(bert_1_run_id)
mlflow.artifacts.download_artifacts(
    f"{bert_1_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bert_1"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename("./src/service/models/bert_1")

03:03:48 INFO:
bert:
  model: bert-base-uncased
  type: BERT
  max_len: 110
  train_batch_size: 32
  validation_batch_size: 32
  number_epochs: 5
  learning_rate_bert: 2.0e-05
  learning_rate_classifier: 0.01
  weight_decay: 0.01
  weighted_classes: false
experiment:
  name: 'Production: BERT_1 Train for Staged Application'
  description: ''
  random_state: 125
  folds: 5
  iterations: 1
  average: macro
  dataset: prolific
  lower_case: false
  force: false
transformation:
  description: Levels
  type: Reduced
  task: Domain_Level
  goals: null
  domain_data: Domain_Level
  activity: Domain_Level
  stakeholder: Domain_Level
  system_function: Interaction_Level
  interaction: Interaction_Level
  interaction_data: Interaction_Level
  workspace: Interaction_Level
  software: System_Level
  internal_action: System_Level
  internal_data: System_Level

03:03:48 INFO:New experiment. Running
03:03:48 INFO:Entering mlflow context
03:03:49 INFO:Using device: mps
03:03:49 INFO:Importing dataset:

Map:   0%|          | 0/1075 [00:00<?, ? examples/s]

Map:   0%|          | 0/269 [00:00<?, ? examples/s]

03:04:23 INFO:Logged iteration result res.precision=0.8544484579644434 res.recall=0.7748789828781871


{'eval_loss': 0.28557825088500977, 'eval_step': 0, 'eval_precision': 0.8544484579644434, 'eval_recall': 0.7748789828781871, 'eval_label_count': 4, 'eval_runtime': 2.2206, 'eval_samples_per_second': 121.14, 'eval_steps_per_second': 4.053, 'epoch': 1.0}


...

03:06:20 INFO:Logged iteration result res.precision=0.8494281543896238 res.recall=0.8229801390592925


{'eval_loss': 0.2659144103527069, 'eval_step': 8, 'eval_precision': 0.8494281543896238, 'eval_recall': 0.8229801390592925, 'eval_label_count': 4, 'eval_runtime': 2.0603, 'eval_samples_per_second': 130.562, 'eval_steps_per_second': 4.368, 'epoch': 5.0}
{'train_runtime': 148.1275, 'train_samples_per_second': 36.286, 'train_steps_per_second': 1.148, 'train_loss': 0.22822191574994255, 'epoch': 5.0}


03:06:26 INFO:Logged iteration result res.precision=0.8546933418222866 res.recall=0.814459573814639
03:06:26 INFO:Logged iteration result res.precision=0.8546933418222866 res.recall=0.814459573814639
03:06:26 INFO:Finished iteration=0
03:06:26 INFO:Logging model artifact (might take a while)
03:08:01 INFO:Breaking early after iteration=0 of 5 folds
03:08:02 INFO:Logged experiment result res.mean_precision=0.8546933418222866 res.mean_recall=0.814459573814639
03:08:02 INFO:Left mlflow context


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
f403ed360ab84762857c70de2ab3c26f


PosixPath('src/service/models/bert_1')

## Train BERT Second Stage Model for BERT First Stage

In [9]:
run_experiments

True

In [10]:
from tooling.observability import get_run_id
from tooling.config import DualModelStagedBERTConfig, StagedBERT

from copy import deepcopy

bert_2_bert_experiment_config = deepcopy(base_experiment_config)
bert_2_bert_experiment_config.name = "Production: BERT_2 Train for Staged Application"

bert_2_bert_config = StagedBERT(max_len=sentence_length)

bert_2_bert_cfg: DualModelStagedBERTConfig = OmegaConf.structured(
    DualModelStagedBERTConfig(
        bert=bert_2_bert_config,
        experiment=bert_2_bert_experiment_config,
        transformation=label_transformation_config,
        first_model_bert=bert_1_cfg,
    )
)

if run_experiments:
    from experiments.dual_model_staged_bert import dual_stage_bert
    dual_stage_bert(OmegaConf.create(bert_2_bert_cfg))

bert_2_bert_run_id = get_run_id(bert_2_bert_cfg, pin_commit = pin_commits)

print(bert_2_bert_run_id)

bert_2_bert_run = mlflow.get_run(bert_2_bert_run_id)
mlflow.artifacts.download_artifacts(
    f"{bert_2_bert_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bert_2_bert"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename("./src/service/models/bert_2_bert")

03:08:19 INFO:
first_model_bert:
  bert:
    model: bert-base-uncased
    type: BERT
    max_len: 110
    train_batch_size: 32
    validation_batch_size: 32
    number_epochs: 5
    learning_rate_bert: 2.0e-05
    learning_rate_classifier: 0.01
    weight_decay: 0.01
    weighted_classes: false
  experiment:
    name: 'Production: BERT_1 Train for Staged Application'
    description: ''
    random_state: 125
    folds: 5
    iterations: 1
    average: macro
    dataset: prolific
    lower_case: false
    force: false
  transformation:
    description: Levels
    type: Reduced
    task: Domain_Level
    goals: null
    domain_data: Domain_Level
    activity: Domain_Level
    stakeholder: Domain_Level
    system_function: Interaction_Level
    interaction: Interaction_Level
    interaction_data: Interaction_Level
    workspace: Interaction_Level
    software: System_Level
    internal_action: System_Level
    internal_data: System_Level
first_model_bilstm: null
first_model_sner: null
ber

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


03:08:20 INFO:Using device: mps
03:08:20 INFO:Found existing run with run_id: f403ed360ab84762857c70de2ab3c26f matching the configuration
03:08:20 INFO:Downloading run model from mlflow-artifacts:/34/f403ed360ab84762857c70de2ab3c26f/artifacts/0_model


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


03:08:38 INFO:Importing dataset: prolific from /Users/bockstaller/code/uvl-tore-classifier-bert/src/data/datasets/prolific/TORE_Coded_Answers_1_33.json
03:08:38 INFO:Importing dataset: prolific from /Users/bockstaller/code/uvl-tore-classifier-bert/src/data/datasets/prolific/TORE_Coded_Answers_34_66.json
03:08:39 INFO:Importing dataset: prolific from /Users/bockstaller/code/uvl-tore-classifier-bert/src/data/datasets/prolific/TORE_Coded_Answers_67_100.json
03:08:41 INFO:Dataset Labels: transformed_dataset['labels']=['Activity', 'Task', 'Interaction', 'Domain_Data', 'Stakeholder', 'System_Level', '0', 'Interaction_Data', 'System_Function', 'Workspace']
03:08:41 INFO:Class weights: {'0': 1.0, 'Task': 1.0, 'Domain_Data': 1.0, 'Activity': 1.0, 'Stakeholder': 1.0, 'System_Function': 1.0, 'Interaction': 1.0, 'Interaction_Data': 1.0, 'Workspace': 1.0, 'System_Level': 1.0}
03:08:41 INFO:Configured maximal token sequence length: max_len = 110
03:08:43 INFO:Created fold datasets for fold: 0, store

...

03:11:53 INFO:Logged iteration result res.precision=0.6977754424758543 res.recall=0.6524043185844804


{'eval_loss': 0.36366942524909973, 'eval_step': 8, 'eval_precision': 0.6977754424758543, 'eval_recall': 0.6524043185844804, 'eval_label_count': 10, 'eval_runtime': 2.1102, 'eval_samples_per_second': 127.474, 'eval_steps_per_second': 4.265, 'epoch': 5.0}
{'train_runtime': 147.9124, 'train_samples_per_second': 36.339, 'train_steps_per_second': 1.149, 'train_loss': 0.30330666934742645, 'epoch': 5.0}


03:11:59 INFO:Logged iteration result res.precision=0.7253131652617635 res.recall=0.6769705769319452
03:11:59 INFO:Logged iteration result res.precision=0.7253131652617635 res.recall=0.6769705769319452
03:11:59 INFO:Finished iteration=0
03:11:59 INFO:Logging model artifact (might take a while)
03:13:38 INFO:Breaking early after iteration=0 of 5 folds
03:13:39 INFO:Logged experiment result res.mean_precision=0.7253131652617635 res.mean_recall=0.6769705769319452
03:13:39 INFO:Left mlflow context


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
220b9b3b1da94ca6a16a23e0b1da5807


PosixPath('src/service/models/bert_2_bert')

## Train BERT Second Stage Model for SNER First Stage

In [11]:
from tooling.observability import get_run_id
from tooling.config import DualModelStagedBERTConfig, StagedBERT

from copy import deepcopy

bert_2_sner_experiment_config = deepcopy(base_experiment_config)
bert_2_sner_experiment_config.name = "Production: BERT_2 Train for Staged Application"

bert_2_sner_config = StagedBERT(max_len=sentence_length)

bert_2_sner_cfg = OmegaConf.structured(
    DualModelStagedBERTConfig(
        bert=bert_2_sner_config,
        experiment=bert_2_sner_experiment_config,
        transformation=label_transformation_config,
        first_model_sner=sner_cfg,
    )
)

if run_experiments:
    from experiments.dual_model_staged_bert import dual_stage_bert
    dual_stage_bert(OmegaConf.create(bert_2_sner_cfg))

bert_2_sner_run_id = get_run_id(bert_2_sner_cfg, pin_commit = pin_commits)

print(bert_2_sner_run_id)

bert_2_sner_run = mlflow.get_run(bert_2_sner_run_id)
mlflow.artifacts.download_artifacts(
    f"{bert_2_sner_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bert_2_sner"))
except FileNotFoundError:
    pass

Path("./src/service/models/0_model").rename("./src/service/models/bert_2_sner")

03:13:56 INFO:
first_model_bert: null
first_model_bilstm: null
first_model_sner:
  sner:
    type: SNER
  experiment:
    name: 'Production: SNER Train for Staged Application'
    description: ''
    random_state: 125
    folds: 5
    iterations: 1
    average: macro
    dataset: prolific
    lower_case: false
    force: false
  transformation:
    description: Levels
    type: Reduced
    task: Domain_Level
    goals: null
    domain_data: Domain_Level
    activity: Domain_Level
    stakeholder: Domain_Level
    system_function: Interaction_Level
    interaction: Interaction_Level
    interaction_data: Interaction_Level
    workspace: Interaction_Level
    software: System_Level
    internal_action: System_Level
    internal_data: System_Level
bert:
  model: bert-base-uncased
  type: BERT
  max_len: 110
  train_batch_size: 32
  validation_batch_size: 32
  number_epochs: 5
  learning_rate_bert: 2.0e-05
  learning_rate_classifier: 0.01
  weight_decay: 0.01
  weighted_classes: false
  la

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


03:13:57 INFO:Using device: mps
03:13:57 INFO:Found existing run with run_id: 5ffff0522cb140cb85d6c0a402b2bd43 matching the configuration
03:13:57 INFO:Downloading run model from mlflow-artifacts:/33/5ffff0522cb140cb85d6c0a402b2bd43/artifacts/0_model.ser.gz


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


03:13:58 INFO:Importing dataset: prolific from /Users/bockstaller/code/uvl-tore-classifier-bert/src/data/datasets/prolific/TORE_Coded_Answers_1_33.json
03:13:59 INFO:Importing dataset: prolific from /Users/bockstaller/code/uvl-tore-classifier-bert/src/data/datasets/prolific/TORE_Coded_Answers_34_66.json
03:13:59 INFO:Importing dataset: prolific from /Users/bockstaller/code/uvl-tore-classifier-bert/src/data/datasets/prolific/TORE_Coded_Answers_67_100.json
03:14:01 INFO:Dataset Labels: transformed_dataset['labels']=['Activity', 'Task', 'Interaction', 'Domain_Data', 'Stakeholder', 'System_Level', '0', 'Interaction_Data', 'System_Function', 'Workspace']
03:14:01 INFO:Class weights: {'0': 1.0, 'Task': 1.0, 'Domain_Data': 1.0, 'Activity': 1.0, 'Stakeholder': 1.0, 'System_Function': 1.0, 'Interaction': 1.0, 'Interaction_Data': 1.0, 'Workspace': 1.0, 'System_Level': 1.0}
03:14:01 INFO:Configured maximal token sequence length: max_len = 110
03:14:03 INFO:Created fold datasets for fold: 0, store

...

03:16:32 INFO:Logged iteration result res.precision=0.6856046373502331 res.recall=0.631763855984542


{'eval_loss': 0.36506086587905884, 'eval_step': 8, 'eval_precision': 0.6856046373502331, 'eval_recall': 0.631763855984542, 'eval_label_count': 10, 'eval_runtime': 2.0793, 'eval_samples_per_second': 129.371, 'eval_steps_per_second': 4.328, 'epoch': 5.0}
{'train_runtime': 146.8901, 'train_samples_per_second': 36.592, 'train_steps_per_second': 1.157, 'train_loss': 0.24524720135857078, 'epoch': 5.0}


03:16:37 INFO:Logged iteration result res.precision=0.7206604419047202 res.recall=0.6598699192764179
03:16:38 INFO:Logged iteration result res.precision=0.7206604419047202 res.recall=0.6598699192764179
03:16:38 INFO:Finished iteration=0
03:16:38 INFO:Logging model artifact (might take a while)
03:18:14 INFO:Breaking early after iteration=0 of 5 folds
03:18:15 INFO:Logged experiment result res.mean_precision=0.7206604419047202 res.mean_recall=0.6598699192764179
03:18:15 INFO:Left mlflow context


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
c238264bed8e400a8ef4bf9f2e5d47e5


PosixPath('src/service/models/bert_2_sner')

## Train BERT Second Stage Model for BILSTM First Stage

In [12]:
from tooling.observability import get_run_id
from tooling.config import DualModelStagedBERTConfig, StagedBERT

from copy import deepcopy

bert_2_bilstm_experiment_config = deepcopy(base_experiment_config)
bert_2_bilstm_experiment_config.name = "Production: BERT_2 Train for Staged Application"

bert_2_bilstm_config = StagedBERT(max_len=sentence_length)

bert_2_bilstm_cfg = OmegaConf.structured(
    DualModelStagedBERTConfig(
        bert=bert_2_bilstm_config,
        experiment=bert_2_bilstm_experiment_config,
        transformation=label_transformation_config,
        first_model_bilstm=bilstm_cfg,
    )
)

if run_experiments:
    from experiments.dual_model_staged_bert import dual_stage_bert
    dual_stage_bert(OmegaConf.create(bert_2_bilstm_cfg))

bert_2_bilstm_run_id = get_run_id(bert_2_bilstm_cfg, pin_commit = pin_commits)

print(bert_2_bilstm_run_id)

bert_2_bilstm_run = mlflow.get_run(bert_2_bilstm_run_id)
mlflow.artifacts.download_artifacts(
    f"{bert_2_bilstm_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bert_2_bilstm"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename(
    "./src/service/models/bert_2_bilstm"
)

03:18:34 INFO:
first_model_bert: null
first_model_bilstm:
  bilstm:
    type: BiLSTM
    sentence_length: 110
    batch_size: 32
    number_epochs: 4
    verbose: 1
    weighted_classes: false
    learning_rate: 0.0001
  experiment:
    name: 'Production: BiLSTM Train for Staged Application'
    description: ''
    random_state: 125
    folds: 5
    iterations: 1
    average: macro
    dataset: prolific
    lower_case: false
    force: false
  transformation:
    description: Levels
    type: Reduced
    task: Domain_Level
    goals: null
    domain_data: Domain_Level
    activity: Domain_Level
    stakeholder: Domain_Level
    system_function: Interaction_Level
    interaction: Interaction_Level
    interaction_data: Interaction_Level
    workspace: Interaction_Level
    software: System_Level
    internal_action: System_Level
    internal_data: System_Level
first_model_sner: null
bert:
  model: bert-base-uncased
  type: BERT
  max_len: 110
  train_batch_size: 32
  validation_batch_si

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


03:18:35 INFO:Using device: mps
03:18:35 INFO:Found existing run with run_id: 941f08130ff84d11bf2c80fc13790a81 matching the configuration
03:18:35 INFO:Downloading run model from mlflow-artifacts:/32/941f08130ff84d11bf2c80fc13790a81/artifacts/0_model


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


03:18:38 INFO:loading projection weights from /Users/bockstaller/gensim-data/glove-twitter-100/glove-twitter-100.gz
03:19:12 INFO:KeyedVectors lifecycle event {'msg': 'loaded (1193514, 100) matrix of type float32 from /Users/bockstaller/gensim-data/glove-twitter-100/glove-twitter-100.gz', 'binary': False, 'encoding': 'utf8', 'datetime': '2023-08-15T15:19:12.741889', 'gensim': '4.3.1', 'python': '3.11.4 (main, Jun 20 2023, 19:14:10) [Clang 14.0.3 (clang-1403.0.22.14.1)]', 'platform': 'macOS-13.3-arm64-arm-64bit', 'event': 'load_word2vec_format'}
03:19:12 INFO:Importing dataset: prolific from /Users/bockstaller/code/uvl-tore-classifier-bert/src/data/datasets/prolific/TORE_Coded_Answers_1_33.json
03:19:13 INFO:Importing dataset: prolific from /Users/bockstaller/code/uvl-tore-classifier-bert/src/data/datasets/prolific/TORE_Coded_Answers_34_66.json
03:19:13 INFO:Importing dataset: prolific from /Users/bockstaller/code/uvl-tore-classifier-bert/src/data/datasets/prolific/TORE_Coded_Answers_67

...

03:22:09 INFO:Logged iteration result res.precision=0.7034568204569754 res.recall=0.6549527130478034


{'eval_loss': 0.362361878156662, 'eval_step': 8, 'eval_precision': 0.7034568204569754, 'eval_recall': 0.6549527130478034, 'eval_label_count': 10, 'eval_runtime': 2.0926, 'eval_samples_per_second': 128.549, 'eval_steps_per_second': 4.301, 'epoch': 5.0}
{'train_runtime': 147.5346, 'train_samples_per_second': 36.432, 'train_steps_per_second': 1.152, 'train_loss': 0.31862718918744254, 'epoch': 5.0}


03:22:15 INFO:Logged iteration result res.precision=0.7153252194444211 res.recall=0.6797689227191499
03:22:15 INFO:Logged iteration result res.precision=0.7153252194444211 res.recall=0.6797689227191499
03:22:15 INFO:Finished iteration=0
03:22:15 INFO:Logging model artifact (might take a while)
03:23:52 INFO:Breaking early after iteration=0 of 5 folds
03:23:53 INFO:Logged experiment result res.mean_precision=0.7153252194444211 res.mean_recall=0.6797689227191499
03:23:53 INFO:Left mlflow context


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
4ece27d8e9c744df81556d3973aab532


PosixPath('src/service/models/bert_2_bilstm')

## Train BERT E2E Model

In [13]:
from tooling.observability import get_run_id
from tooling.config import BERTConfig, BERT

from copy import deepcopy

bert_experiment_config = deepcopy(base_experiment_config)
bert_experiment_config.name = "Production: BERT Train for E2E Application"

bert_config = BERT(max_len=sentence_length)

bert_cfg = OmegaConf.structured(
    BERTConfig(
        bert=bert_config,
        experiment=bert_experiment_config,
        transformation=label_transformation_config,
    )
)

if run_experiments:
    from experiments.bert import bert
    bert(OmegaConf.create(bert_cfg))

bert_run_id = get_run_id(bert_cfg, pin_commit = pin_commits)

print(bert_run_id)

run = mlflow.get_run(bert_run_id)
mlflow.artifacts.download_artifacts(
    f"{run.info.artifact_uri}/0_model", dst_path=Path("./src/service/models/")
)
try:
    shutil.rmtree(Path("./src/service/models/bert"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename("./src/service/models/bert")

03:24:12 INFO:
bert:
  model: bert-base-uncased
  type: BERT
  max_len: 110
  train_batch_size: 32
  validation_batch_size: 32
  number_epochs: 5
  learning_rate_bert: 2.0e-05
  learning_rate_classifier: 0.01
  weight_decay: 0.01
  weighted_classes: false
experiment:
  name: 'Production: BERT Train for E2E Application'
  description: ''
  random_state: 125
  folds: 5
  iterations: 1
  average: macro
  dataset: prolific
  lower_case: false
  force: false
transformation:
  description: None
  type: Full
  task: Task
  goals: null
  domain_data: Domain_Data
  activity: Activity
  stakeholder: Stakeholder
  system_function: System_Function
  interaction: Interaction
  interaction_data: Interaction_Data
  workspace: Workspace
  software: System_Level
  internal_action: System_Level
  internal_data: System_Level

03:24:12 INFO:New experiment. Running
03:24:12 INFO:Entering mlflow context


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


03:24:12 INFO:Using device: mps
03:24:12 INFO:Importing dataset: prolific from /Users/bockstaller/code/uvl-tore-classifier-bert/src/data/datasets/prolific/TORE_Coded_Answers_1_33.json
03:24:13 INFO:Importing dataset: prolific from /Users/bockstaller/code/uvl-tore-classifier-bert/src/data/datasets/prolific/TORE_Coded_Answers_34_66.json
03:24:13 INFO:Importing dataset: prolific from /Users/bockstaller/code/uvl-tore-classifier-bert/src/data/datasets/prolific/TORE_Coded_Answers_67_100.json
03:24:15 INFO:Dataset Labels: transformed_dataset['labels']=['Activity', 'Task', 'Interaction', 'Domain_Data', 'Stakeholder', 'System_Level', '0', 'Interaction_Data', 'System_Function', 'Workspace']
03:24:15 INFO:Class weights: {'0': 1.0, 'Task': 1.0, 'Domain_Data': 1.0, 'Activity': 1.0, 'Stakeholder': 1.0, 'System_Function': 1.0, 'Interaction': 1.0, 'Interaction_Data': 1.0, 'Workspace': 1.0, 'System_Level': 1.0}
03:24:15 INFO:Configured maximal token sequence length: max_len = 110
03:24:17 INFO:Created 

Map:   0%|          | 0/1075 [00:00<?, ? examples/s]

Map:   0%|          | 0/269 [00:00<?, ? examples/s]

...

03:26:41 INFO:Logged iteration result res.precision=0.6996668614582978 res.recall=0.6556065767081019


{'eval_loss': 0.3600035607814789, 'eval_step': 8, 'eval_precision': 0.6996668614582978, 'eval_recall': 0.6556065767081019, 'eval_label_count': 10, 'eval_runtime': 2.1107, 'eval_samples_per_second': 127.443, 'eval_steps_per_second': 4.264, 'epoch': 5.0}
{'train_runtime': 145.6476, 'train_samples_per_second': 36.904, 'train_steps_per_second': 1.167, 'train_loss': 0.3214408874511719, 'epoch': 5.0}


03:26:47 INFO:Logged iteration result res.precision=0.7178643892293535 res.recall=0.6810540380395174
03:26:47 INFO:Logged iteration result res.precision=0.7178643892293535 res.recall=0.6810540380395174
03:26:47 INFO:Finished iteration=0
03:26:47 INFO:Logging model artifact (might take a while)
03:28:23 INFO:Breaking early after iteration=0 of 5 folds
03:28:24 INFO:Logged experiment result res.mean_precision=0.7178643892293535 res.mean_recall=0.6810540380395174
03:28:24 INFO:Left mlflow context


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
6552cb782d6e4d1686666fd66e69e09a


PosixPath('src/service/models/bert')